### Gruppo 5.a: Castagnacci Giulia (581749), Giordano Elisabetta (536265)

### Analisi fragmentlines

In [1]:

using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using BenchmarkTools
using OrderedCollections
using Base.Threads


### Dati in input

In [2]:
V = hcat([[0.,0],[1,0],[1,1],[0,1],[2,1]]...);                                          #vertici del modello 2D
V3 = hcat([[0.,0,0],[1,0,3],[1,1,2],[0,1,1],[2,1,0]]...);                               #vertici del modello 3D
EV = [[1,2],[2,3],[3,4],[4,1],[1,5]];                                                   #spigoli del modello
bb = [[0.0 1.0; 0.0 0.0], [1.0 1.0; 0.0 1.0], [0.0 1.0; 1.0 1.0], [0.0 0.0; 0.0 1.0], [0.0 2.0; 0.0 1.0]];  #bounding box
dict = OrderedDict([0.0, 1.0] => [1, 3],[1.0, 1.0] => [2],[0.0, 0.0] => [4],[0.0, 2.0] => [5])  #dizionario intervallo/indice
cov = [[4, 1, 3, 5, 2], [1, 3, 5, 2], [4, 1, 3, 5, 2], [4, 1, 3, 5], [4, 1, 3, 5, 2]]    #intersezioni tra bounding box

5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

### Funzioni di supporto

In [3]:
function spaceindex(model::Lar.LAR)::Array{Array{Int,1},1}
	V,CV = model[1:2]
	dim = size(V,1)
	cellpoints = [ V[:,CV[k]]::Lar.Points for k=1:length(CV) ]
	#----------------------------------------------------------
	bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
	xboxdict = Lar.coordintervals(1,bboxes)
	yboxdict = Lar.coordintervals(2,bboxes)
	# xs,ys are IntervalTree type
	xs = IntervalTrees.IntervalMap{Float64, Array}()
	for (key,boxset) in xboxdict
		xs[tuple(key...)] = boxset
	end
	ys = IntervalTrees.IntervalMap{Float64, Array}()
	for (key,boxset) in yboxdict
		ys[tuple(key...)] = boxset
	end
	xcovers = Lar.boxcovering(bboxes, 1, xs)
	ycovers = Lar.boxcovering(bboxes, 2, ys)
	covers = [intersect(pair...) for pair in zip(xcovers,ycovers)]

	if dim == 3
		zboxdict = Lar.coordintervals(3,bboxes)
		zs = IntervalTrees.IntervalMap{Float64, Array}()
		for (key,boxset) in zboxdict
			zs[tuple(key...)] = boxset
		end
		zcovers = Lar.boxcovering(bboxes, 3, zs)
		covers = [intersect(pair...) for pair in zip(zcovers,covers)]
	end
	# remove each cell from its cover
	for k=1:length(covers)
		covers[k] = setdiff(covers[k],[k])
	end
	return covers
end


function boundingbox(vertices::Lar.Points)
    minimum = mapslices(x->min(x...), vertices, dims=2)
    maximum = mapslices(x->max(x...), vertices, dims=2)
    return minimum, maximum
 end


 function boxcovering(bboxes, index, tree)
	covers = [[] for k=1:length(bboxes)]
	for (i,boundingbox) in enumerate(bboxes)
		extent = bboxes[i][index,:]
		iterator = IntervalTrees.intersect(tree, tuple(extent...))
		for x in iterator
			append!(covers[i],x.value)
		end
	end
	return covers
end

function linefragments(V,EV,Sigma)
	# remove the double intersections by ordering Sigma
	m = length(Sigma)
	sigma = map(sort,Sigma)
	reducedsigma = sigma ##[filter(x->(x > k), sigma[k]) for k=1:m]
	# pairwise parametric intersection
	params = Array{Float64,1}[[] for i=1:m]
	for h=1:m
		if sigma[h] ≠ []
			line1 = V[:,EV[h]]
			for k in sigma[h]
				line2 = V[:,EV[k]]
				out = Lar.intersection(line1,line2) # TODO: w interval arithmetic
				if out ≠ nothing
					α,β = out
					if 0<=α<=1 && 0<=β<=1
						push!(params[h], α)
						push!(params[k], β)
					end
				end
			end
		end
	end
	# finalize parameters of fragmented lines
	fragparams = []
	for line in params
		push!(line, 0.0, 1.0)
		line = sort(collect(Set(line)))
		push!(fragparams, line)
	end
	return fragparams
end


function intersection(line1,line2)
	x1,y1,x2,y2 = vcat(line1...)
	x3,y3,x4,y4 = vcat(line2...)

	# intersect lines e1,e2
	det = (x4-x3)*(y1-y2)-(x1-x2)*(y4-y3)
	if det != 0.0
		a = 1/det
		b = [y1-y2 x2-x1; y3-y4 x4-x3]
		c = [x1-x3; y1-y3]
		(β,α) = a * b * c
	else
		if (y1==y2) == (y3==y4) || (x1==x2) == (x3==x4) # segments collinear
			 return nothing
		else
			 # segments parallel: no intersection
			 return nothing
		end
	end
	return α,β
end


function congruence(model)
	W,EW = model
	# congruent vertices
	balltree = NearestNeighbors.BallTree(W)
	r = 0.0000000001
	near = Array{Any}(undef, size(W,2))
	for k=1:size(W,2)
		near[k] = union([NearestNeighbors.inrange(balltree, W[:,k], r, true)]...)
	end
	near = map(sort,near) 
	for k=1:size(W,2)
		W[:,k] = W[:,near[k][1]]
	end
	pointidx = [ near[k][1] for k=1:size(W,2) ]  # check !!
	invidx = OrderedDict(zip(1:length(pointidx), pointidx))
	V = [W[:,k] for k=1:length(pointidx)]
	# congruent edges
	EV = []
	for e in (EW)
		newedge = [invidx[e[1]],invidx[e[2]]]
		if newedge[1] !== newedge[2]
			push!(EV,newedge)
		end
	end
	EV = [EV[h] for h=1:length(EV) if length(EV[h])==2]
	EV = convert(Lar.Cells, EV)
	
	return hcat(V...),EV
end

function approxVal(PRECISION)
    function approxVal0(value)
        out = round(value, digits=PRECISION)
        if out == -0.0
            out = 0.0
        end
        return out
    end
    return approxVal0
end

approxVal (generic function with 1 method)

### Versione iniziale fragmentlines

Funzione utilizzata per l'intersezione di coppie di segmenti di linea. In particolare, crea un indice spaziale "Sigma", calcola i parametri d'intersezione degli spigoli e restituisce i nuovi punti generati dall'intersezione tra spigoli, tramite i parametri d'intersezione, all'interno di un array. Per N punti d'intersezione trovati, vengono generati N-1 spigoli.

In [4]:
function fragmentlines(model)
	V,EV = model
	# acceleration via spatial index computation
	Sigma = spaceindex(model)
	# actual parametric intersection of each line with the close ones
	lineparams = linefragments(V,EV,Sigma)
	# initialization of local data structures
	vertdict = OrderedDict{Array{Float64,1},Array{Int,1}}()
	pairs = collect(zip(lineparams, [V[:,e] for e in EV]))
	vertdict = OrderedDict{Array{Float64,1},Int}()
	W = Array[]
	EW = Array[]
	k = 0
	# generation of intersection points
	for (params,linepoints) in pairs
		v1 = linepoints[:,1]
		v2 = linepoints[:,2]
		points = [ v1 + t*(v2 - v1) for t in params]
		vs = zeros(Int64,1,length(points))
		PRECISION = 8
		# identification via dictionary of points
		for (h,point) in enumerate(points)
			point = map(approxVal(PRECISION), point)
			if haskey(vertdict, point) == false
				k += 1
				vertdict[point] = k
				push!(W, point)
			end
			vs[h] = vertdict[point]
		end
		[push!(EW, [vs[k], vs[k+1]]) for k=1:length(vs)-1]
	end
	# normalization of output
	W,EW = hcat(W...),convert(Array{Array{Int64,1},1},EW)
	V,EV = congruence((W,EW))
	return V,EV
end

fragmentlines (generic function with 1 method)

### Analisi del comportamento e dei tempi della versione iniziale

In [5]:
@btime fragmentlines((V,EV))

  332.900 μs (2334 allocations: 97.78 KiB)


([0.0 1.0 … 0.0 2.0; 0.0 0.0 … 1.0 1.0], [[1, 2], [2, 3], [3, 4], [4, 5], [5, 1], [1, 3], [3, 6]])

In [6]:
@code_warntype fragmentlines((V,EV))

MethodInstance for 

fragmentlines(::

Tuple{Matrix{Float64}, Vector{Vector{Int64}}})
  from fragmentlines(model) in Main at c:\Users\giord\eclipse-SIW\LARSplitting2D\notebooks\refactoringAnalisi_fragmentlines.ipynb:1


Arguments
  

#self#::Core.Const(fragmentlines)
  model::Tuple{Matrix{Float64}, Vector{Vector{Int64}}}
Locals
  @_3::Int64
  @_4::Any
  #32::var"#32#35"
  @_6::Int64
  k::Int64
  EW@_8::Core.Box
  W::Any
  pairs::Any
  vertdict::Union{OrderedDict{Vector{Float64}, Vector{Int64}}, OrderedDict{Vector{Float64}, Int64}}
  lineparams::Vector{Any}
  Sigma::Vector{Vector{Int64}}
  EV::Any
  V@_15::Core.Box
  #34::var"#34#37"{Matrix{Int64}}
  @_17::Union{Nothing, Tuple{Tuple{Int64, Any}, Tuple{Int64, Any}}}
  #33::var"#33#36"
  @_19::Any
  linepoints::Any
  params::Any
  PRECISION::Int64
  vs::Matrix{Int64}
  points::Any
  v2::Any
  v1::Any
  @_27::Int64
  point::Any
  h::Int64
  V@_30::Union{}
  EW@_31::Union{}
  EW@_32::Union{}
  V@_33::Union{}
Body::Tuple{Any, Any}

...

false)::Any
└───        goto #8 if not %88
7 ──        (k = k + 1)
│           Base.setindex!(vertdict::OrderedDict{Vector{Float64}, Int64}, k, point)
└───        Main.push!(W::Vector{Array}, point)
8 ┄─ %93  = Base.getindex(vertdict::OrderedDict{Vector{Float64}, Int64}, point)::Int64
│           Base.setindex!(vs, %93, h)
│           (@_17 = Base.iterate(%72, %84))
│    %96  = (@_17 === nothing)::Bool
│    %97  = Base.not_int(%96)::Bool
└───        goto #10 if not %97
9 ──        goto #6
10 ┄ %100 = Main.:(var"#34#37")::Core.Const(var"#34#37")
│    %101 = Core.typeof(vs)::Core.Const(Matrix{Int64})
│    %102 = Core.apply_type(%100, %101)

::Core.Const(var"#34#37"{Matrix{Int64}})
│    %103 = EW@_8::Core.Box
│           (#34 = %new(%102, %103, vs))
│    %105 = #34::var"#34#37"{Matrix{Int64}}
│    %106 = Main.length(vs)::Int64
│    %107 = (%106 - 1)::Int64
│    %108 = (1:%107)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│    %109 = Base.Generator(%105, %108)

::Core.PartialStruct(Base.Generator{UnitRange{Int64}, var"#34#37"{Matrix{Int64}}}, Any[var"#34#37"{Matrix{Int64}}, Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])])
│           Base.collect(%109)
│           (@_4 = Base.iterate(%44, %57))
│    %112 = (@_4 === nothing)::Bool
│    %113 = Base.not_int(%112)::Bool
└───        goto #12 if not %113
11 ─        goto #5
12 ┄ %116 = Core._apply_iterate(Base.iterate, Main.hcat, W::Vector{Array})::Any
│    %117 = Core.apply_type(Main.Array, Main.Int64, 1)::Core.Const(Vector{Int64})
│    %118 = Core.apply_type(Main.Array, %117, 1)::Core.Const(Vector{Vector{Int64}})
│    %119 = Core.isdefined(EW@_8, :contents)::Bool
└───        goto #14 if not %119
13 ─        goto #15
14 ─        Core.NewvarNode(:(EW@_31))
└───        EW@_31
15 ┄ %124 = Core.

getfield(EW@_8, :contents)::Any
│    %125 = Main.convert(%118, %124)::Any
│           (W = %116)
│           Core.setfield!(EW@_8, :contents, %125)
│    %128 = W::Any
│    %129 = Core.isdefined(EW@_8, :contents)::Bool
└───        goto #17 if not %129
16 ─        goto #18
17 ─        Core.NewvarNode(:(EW@_32))
└───        EW@_32
18 ┄ %134 = Core.getfield(EW@_8, :contents)::Any
│    %135 = Core.tuple(%128, %134)::Tuple{Any, Any}
│    %136 = Main.congruence(%135)::Tuple{Any, Any}
│    %137 = Base.indexed_iterate(%136, 1)::Core.PartialStruct(Tuple{Any, Int64}, Any[Any, Core.Const(2)])
│    %138 = Core.getfield(%137, 1)::Any
│           Core.setfield!(V@_15, :

contents, %138)
│           (@_3 = Core.getfield(%137, 2))
│    %141 = Base.indexed_iterate(%136, 2, @_3::Core.Const(2))::Core.PartialStruct(Tuple{Any, Int64}, Any[Any, Core.Const(3)])
│           (EV = Core.getfield(%141, 1))
│    %143 = Core.isdefined(V@_15, :contents)::Bool
└───        goto #20 if not %143
19 ─        goto #21
20 ─        Core.NewvarNode(:(V@_33))
└───        V@_33
21 ┄ %148 = Core.getfield(V@_15, :contents)::Any
│    %149 = Core.tuple(%148, EV)::Tuple{Any, Any}
└───        return %149



In [7]:
@benchmark fragmentlines((V,EV))

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  336.300 μs …  14.996 ms  ┊ GC (min … max): 0.00% … 94.09%
 Time  (median):     351.800 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   428.779 μs ± 490.104 μs  ┊ GC (mean ± σ):  4.93% ±  4.25%

  ▅█▇▅▄▃▃▂▂▁▃▂▁     ▁▁  ▃▃▁▁             ▃▄▅▄▃▁▁                ▂
  █████████████▇▇▆▇▆██▆▇█████▇▆▇▇▆▄▆▄▃▅▅▆███████████▇▇▇▆▆▄▆▃▄▅▄ █
  336 μs        Histogram: log(frequency) by time        662 μs <

 Memory estimate: 97.78 KiB, allocs estimate: 2334.

### Versione parallelizzata di fragmentlines
Per la funzione fragmentlines abbiamo migliorato le performance con i seguenti passi:
1. Abbiamo convertito la list comprension della creazione dei vettori “params” in un ciclo for in modo da poter utilizzare la macro @threads (in quanto task parallelizzabile)
2. Abbiamo allocato line1 e line2 fuori dal for in modo tale che venissero distrutti e riallocati ad ogni iterazione
3. Abbiamo eliminato la variabile “fragparams” presente nell'ultima porzione di codice, in quanto riallocava la stessa informazione contenuta in 'params' eliminando solo i doppioni. Inoltre abbiamo aggiunto la macro threads a tutte le task parallelizzabili.
4. Abbiamo convertito l'ultimo for each in un for i=1:n


In [8]:
function fragmentlines2(model)
    V,EV = model
    Sigma = spaceindex(model)
    lineparams = linefragments(V,EV,Sigma)
    vertdict = OrderedDict{Array{Float64,1},Array{Int,1}}()
    pairs = collect(zip(lineparams, [V[:,e] for e in EV]))
    vertdict = OrderedDict{Array{Float64,1},Int}()
    W = Array[]
    EW = Array[]
    k = 0
    l = length(pairs)
    @inbounds for i = 1:l
        params = pairs[i][1]
        linepoints = pairs[i][2]
        v1 = linepoints[:,1]    # Isolo primo punto dello spigolo
        v2 = linepoints[:,2]    # Isolo secondo punto dello spigolo
        points = [ v1 + t*(v2 - v1) for t in params] 
        vs = zeros(Int64,1,length(points))
        PRECISION = 8
        numpoint = length(points)
        @inbounds for h = 1:numpoint
            points[h] = map(approxVal(PRECISION), points[h])
            if !haskey(vertdict, points[h])
                k += 1  # Genero ID punto 
                vertdict[points[h]] = k     # Associo l'ID al punto
                push!(W, points[h])     # Effettua una push del punto(x,y) nell'array W
            end
            vs[h] = vertdict[points[h]] 
        end
        m = length(vs) - 1
        @inbounds for k=1:m
            push!(EW, [vs[k], vs[k+1]])
        end
    end
    W,EW = hcat(W...),convert(Array{Array{Int64,1},1},EW)
    V,EV = congruence((W,EW))
    return V,EV
end


fragmentlines2 (generic function with 1 method)

### Analisi del comportamento e dei tempi della versione parallelizzata

In [9]:
@btime fragmentlines2((V,EV))

  313.200 μs (2280 allocations: 95.84 KiB)


([0.0 1.0 … 0.0 2.0; 0.0 0.0 … 1.0 1.0], [[1, 2], [2, 3], [3, 4], [4, 5], [5, 1], [1, 3], [3, 6]])

In [10]:
@benchmark fragmentlines2((V,EV))

BenchmarkTools.Trial: 9470 samples with 1 evaluation.
 Range (min … max):  297.700 μs …  18.274 ms  ┊ GC (min … max): 0.00% … 96.02%
 Time  (median):     451.900 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   518.257 μs ± 639.550 μs  ┊ GC (mean ± σ):  5.00% ±  4.15%

    █▃            ▅                                              
  ▃▅██▅▅▅▄▄▃▅▃▂▂▂▆█▄▃▂▂▂▂▁▃▅▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  298 μs           Histogram: frequency by time         1.25 ms <

 Memory estimate: 95.84 KiB, allocs estimate: 2280.

## Test

In [11]:
using Test

@testset "fragmentlines Tests" begin
	V = hcat([[0.,0],[1,0],[1,1],[0,1],[2,1]]...);
	EV = [[1,2],[2,3],[3,4],[4,1],[1,5]];
	W,EW = Lar.fragmentlines((V,EV))
	@test W == [0.0  1.0  1.0  1.0  0.0  2.0; 0.0  0.0  0.5  1.0  1.0  1.0]
	@test EW == [[1, 2],[2, 3],[3, 4],[4, 5],[5, 1],[1, 3],[3, 6]]
end

Test Summary:       | 

Pass  Total
fragmentlines Tests |    2      2


Test.DefaultTestSet("fragmentlines Tests", Any[], 2, false, false)

![](https://github.com/GiuliaCastagnacci/LARSplitting2D/blob/main/docs/plot/screenTest/test_fragmentlines.png?raw=true)

![](https://github.com/GiuliaCastagnacci/LARSplitting2D/blob/main/docs/plot/images/test/test_fragmentlines.png?raw=true) 